#### Visualization of Xenium add-on panel healthy gut data after the labels transer 
- **Developed by:** Anna Maguza
- **Place:** Wuerzburg Institute for System Immunology
- **Date:** 20th March 2024

##### Import packages

In [ ]:
import anndata as ad
import scanpy as sc
import squidpy as sq
import pandas as pd
from scipy.io import mmread
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
from scipy.sparse import coo_matrix

#### Set up working environment

In [ ]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

#### Data upload

+ Load h5ad object

In [ ]:
adata_xenium = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/10X_Xenium_add-on_without_image.h5ad')

#### Visualise predicted cell types - after cell types + Stem cells labels transfer

+ Upload the object with cell types

In [ ]:
adata_full = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_add_on_data_scANVI_cell_type_and_stem_cells.h5ad')

In [ ]:
adata_full.obs.index.name = 'cell_id'

In [ ]:
adata_xenium.obs.index.name = 'cell_id'

In [ ]:
del adata_xenium.obs['cell_id']

In [ ]:
adata_xenium.obs = adata_xenium.obs.merge(adata_full.obs[['C_scANVI']], on='cell_id', how='left')

In [ ]:
adata_xenium.obs.rename(columns={'C_scANVI':'Cell_Type'}, inplace=True)

+ Visualize the cell types with standard color pallet

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='lowres',
    shape=None,
    color=[
        "Cell_Type",
    ],
    size=0.1,
    alpha=1
)

In [ ]:
adata_xenium.uns['Cell_Type_colors']

+ Change the color pallet to highlight stem cells

In [ ]:
# Define your custom color palette
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells + 
    "#FF0000FF",  # Stem cells + 
    "#ffde17"   # T cells +
]

# Update the color codes in the AnnData object
adata_xenium.uns['Cell_Type_colors'] = custom_colors

# Now, when you plot using sq.pl.spatial_scatter, it should use your updated custom colors
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="Cell_Type",
    size=0.1,
    alpha=1
)

+ Add stem cells types

In [ ]:
adata_stem_cells = sc.read_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/Stem_cells_scRNA_and_Xenium_add_on_scANVI_7K_cells.h5ad')

In [ ]:
adata_stem_cells = adata_stem_cells[adata_stem_cells.obs['Study_name'] == '10X_Xenium_gut_data']

In [ ]:
adata_stem_cells = adata_stem_cells[adata_stem_cells.obs.index.isin(adata_xenium.obs.index)]

In [ ]:
adata_xenium.obs['Stem_cells'] = 'Unknown'
adata_xenium.obs.loc[adata_stem_cells.obs.index, 'Stem_cells'] = adata_stem_cells.obs['C_scANVI']

In [ ]:
adata_xenium.obs['CellType2'] = adata_xenium.obs['Cell_Type'].copy()
adata_xenium.obs['CellType2'] = adata_xenium.obs['CellType2'].cat.add_categories(['MTRNR2L12+ASS1+_SC',
                                                                                  'RPS10+_RPS17+_SC',
                                                                                  'FXYD3+_CKB+_SC'])

adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'MTRNR2L12+ASS1+_SC'), 'CellType2'] = 'MTRNR2L12+ASS1+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'RPS10+_RPS17+_SC'), 'CellType2'] = 'RPS10+_RPS17+_SC'
adata_xenium.obs.loc[(adata_xenium.obs['Stem_cells'] == 'FXYD3+_CKB+_SC'), 'CellType2'] = 'FXYD3+_CKB+_SC'

In [ ]:
adata_xenium.obs['CellType2'] = adata_xenium.obs['CellType2'].cat.remove_unused_categories()

+ Visualize cell types + stem cells populations with the standard color pallet

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(10, 10))
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True, img_res_key='lowres',
    shape=None,
    color=[
        "CellType2",
    ],
    size=0.1,
    alpha=1
)

+ Change color pallete

In [ ]:
# Define your custom color palette
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells +  
    "#ffde17",   # T cells +
    "#00B1B0",  # Stem cells MTRN
    "#508FF7",  # Stem cells RPS
    "#FF0000FF"  # Stem cells FXYD
]

# Update the color codes in the AnnData object
adata_xenium.uns['CellType2_colors'] = custom_colors

# Now, when you plot using sq.pl.spatial_scatter, it should use your updated custom colors
sq.pl.spatial_scatter(
    adata_xenium,
    library_id="spatial",
    img=True,
    img_res_key='lowres',
    shape=None,
    color="CellType2",
    size=0.1,
    alpha=1
)

In [ ]:
adata_xenium.write_h5ad('/mnt/LaCIE/annaM/gut_project/Processed_data/Gut_data/10X_Xenium_data/Xenium_add_on_celltypes_and_stem_cells_types_without_image.h5ad')

+ Visualise only Stem cells

In [ ]:
custom_colors = [
    "#41729F",  # B cells
    "#A16AE8",  # Endothelial
    "#B6E5D8",  # Epithelial
    "#EC9EC0",  # Mesenchymal + 
    "#FBE5C8",  # Myeloid + 
    "#4D5D53",  # Neuronal + 
    "#870052",  # Plasma cells +  
    "#ffde17",   # T cells +
    "#00B1B0",  # Stem cells MTRN
    '#FFDB15',  # Stem cells RPS
    "#FF0000FF"  # Stem cells FXYD
]

# Update the color codes in the AnnData object
adata_xenium.uns['CellType2_colors'] = custom_colors

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, figsize=(20, 10))

cell_types_of_interest = ['MTRNR2L12+ASS1+_SC', 'RPS10+_RPS17+_SC', 'FXYD3+_CKB+_SC']

stem_cells_mask = adata_xenium.obs['CellType2'].isin(cell_types_of_interest)

In [ ]:
adata_stem_cells = adata_xenium[stem_cells_mask, :]

In [ ]:
sq.pl.spatial_scatter(
    adata_stem_cells,  # Use the filtered AnnData object.
    library_id="spatial",
    img=True,
    shape=None,  # Assuming you want the default shape.
    color="CellType2",  # This will now only show 'Stem cells'.
    size=0.1,
    alpha=1
)